In [3]:
%run calibration_Cube_v4.py
%matplotlib

Using matplotlib backend: Qt5Agg


### load data and define parameter

In [4]:
# initial parameter required
# pixel for SZ 10-12
name_RoI = ['Dual+MY', 'Dual+MY 1', 'Dual+MY 2']
name_dyes = ['Dual+MY']
name_single = ['Pt-TFPP+MY', 'Pd-TFPP+MY']
unit = 'hPa'
analyte = 'O2'

# pixel - selection from image cube shape: px(y/x) only dualindicator (name_dyes)
# TFPP
pixel_rot = [[(305, 1070), (400, 1070), (400, 1140), (305, 1140)],
             [(410, 1050), (500, 1050), (500, 1140), (410, 1140)], 
             [(505, 1040), (570, 1040), (570, 1110), (505, 1110)]] 
# TPTBP
#pixel_rot = [[(800, 987), (850, 987), (850, 1120), (800, 1120)],
#             [(900, 980), (990, 980), (990, 1125), (900, 1125)], 
#             [(1042, 1000), (1073, 1000), (1073, 1130), (1042, 1130)]] 
print('Pixel size analyzed:')
print('\t', name_dyes[0], (pixel_rot[0][1][0] - pixel_rot[0][0][0]) * (pixel_rot[0][2][1] - pixel_rot[0][0][1]), 'px')

# ----------------------------------
arg = {'px color': '#a40019', 'color Pt': '#ff8c00', 'color Pd': '#347474', 'colors':'viridis',
       'color dual 1': '#655c56', 'color dual 2': '#6ba8a9', 
       'figure size meas': (5,3), 'fontsize meas': 13.,  # general information for plot
       'rotation': 180., # load testcube -> rotate the cube for better visualization
       'show bands': 120} # which band shall be plotted in testcube?

# information for fitting and linear unmixing
arg_fit = {'fit range sensor': (600, 800), 'fit range ref': (490, 580),
           'fit concentration': (0, 1000), # which concentration points shall be analyzed 
           'range lc': (600, 800)} # crop the region for linear unmixing
bnds = ((0, None), (0, None)) # in order of name_single

# Saving option. Note that the saving of svg-Files requires a rather long time.
save_op = {'type': ['png', 'tiff', 'svg'], 'dpi': 300.}

# directory to measurement / calibration / generic function / reference files
# the calibraiton files (dhr, raw) should contain all information; not only the RoI 
# (of the single indicator)
file_calib_raw = '20200701_ID-SZ10-16_radiometric/TFPP/calibration/Optode_100ms_0hPa_cube_radiometric.hdr'
path_calib = '20200701_ID-SZ10-16_radiometric/TFPP/calibration/' 
file_generic = '20200701_ID-SZ10-16_radiometric/TFPP/output/Generic_function/202086_Generic_function.txt'
path_ref = '20200701_ID-SZ10-16_radiometric/TFPP/Firesting/O2_calibration_100ms.txt'
path_res = '20200701_ID-SZ10-16_radiometric/TFPP/output/'

Pixel size analyzed:
	 Dual+MY 6650 px


#### pre-test for pixel and individual file 

In [4]:
# test plot cube -> check whether selected pixel/area is within cube shape
ls_px = []
for p in pixel_rot:
    for pp in p:
        ls_px.append(pp)
corr.load_testcube(file_hdr=file_calib_raw, pixel_rot=ls_px, span_area=True, arg=arg) 

#### Split all calibration cubes into RoI and store information

In [92]:
start_time = time.time()
cube = [corr.hyperCube_preparation(i, arg=arg, name_dyes=name_RoI, pixel_rot=pixel_rot, unit=unit,
                                   analyte=analyte, averaging=True, plotting=True, save=False, 
                                   cube_type='multiple') 
        for i in glob(path_calib + '/*.hdr')]
print('=================================')
print("Time of execution --- %s seconds " % (time.time() - start_time))

Time of execution --- 19.241759777069092 seconds 


#### calibration dual indicator

In [6]:
%run calibration_Cube_v4.py

# calibration files which are already splitted into RoI
path_dual = '20200701_ID-SZ10-16_radiometric/TFPP/output/correctionCube/multiIndicator/'

par0 = dict({'Pt-TFPP+MY': dict({'k': 0.1144, 'f': 0.35}), 
             'Pd-TFPP+MY': dict({'k': 0.8892, 'f': 0.35})})
#par0 = dict({'Pt-TPTBP+MY': dict({'k': 0.0447, 'f': 0.887}), 
#             'Pd-TPTBP+MY': dict({'k': 0.2132, 'f': 0.887})})

ref_equal = False # assuming that the reference remain unchanged -> average reference (True)
correct_cube = True # does path_dual contain already corrected cubes (True) or not? If not provide the 
                    # path path_calib to the raw cubes so the correction can be conducted 
simply = True

ls_outlier = None # ['485hPa'] # ['75pc', '35pc', '53pc']
save_cube = False
start_time = time.time()
# =================================================================
[dic_av, prefactors, prefactors_std, dic_integral, df_ref_int, 
 dic_SVFit] = dualSensor_calibration_v2(file_generic=file_generic, path_res=path_res, par0=par0,
                                        path_dual=path_dual, ls_outlier=ls_outlier, path_ref=path_ref,
                                        arg=arg, arg_fit=arg_fit, name_dyes=name_RoI, save_op=save_op,
                                        bnds=bnds, ref_equal=ref_equal, corrected_cube=correct_cube,
                                        val_name=name_dyes[0], name_single=name_single, unit=unit,
                                        analyte=analyte, name_ind=name_dyes, lunmix_method='SLSQP',
                                        threshold=15., ratiometric=True, plotting_fit=True,
                                        plot_validation=True, save_res=False, simply=simply)
print('Calibration check')
r = dic_SVFit['Report']
print('\t K(sv) -', name_single[0], round(r[name_dyes[0]][name_single[0]].best_values['k'], 3))
print('\t K(sv) -', name_single[1], round(r[name_dyes[0]][name_single[1]].best_values['k'], 3))
print('\t K(sv) ratio',  [x for x in name_single if x.startswith('Pd')][0], '/',  [x for x in name_single if x.startswith('Pt')][0], 
      round(r[name_dyes[0]][[x for x in name_single if x.startswith('Pd')][0]].best_values['k'] / r[name_dyes[0]][[x for x in name_single if x.startswith('Pt')][0]].best_values['k'], 3))
print('=================================')
print("Time of execution --- %s seconds " % (time.time() - start_time))

WARNING! The spectral deviation of the fit result to the measured data for concentration point 0.00 exceed the default threshold level of 15.00‰
WARNING! The spectral deviation of the fit result to the measured data for concentration point 24.84 exceed the default threshold level of 15.00‰
Calibration check
	 K(sv) - Pt-TFPP+MY 0.012
	 K(sv) - Pd-TFPP+MY 0.07
	 K(sv) ratio Pd-TFPP+MY / Pt-TFPP+MY 5.891
Time of execution --- 15.594842672348022 seconds 
